文件存储文件夹

`csv`是结果文件

`match_frame`是存放要匹配的地名表，数据汇总到了`concat_final.csv`中

`txt`是过渡文件，访谈稿匹配的结果先放在`txt`中，之后再转到`csv`中


# Preparation

In [1]:
import os
import pandas as pd
from docx import Document
from tqdm import tqdm
import ast


In [2]:
# file_list = os.listdir("data/Interview transcripts-round 3")
file_list = ['1_SZUZJY_来深33年邹先生.docx',
 '1_SZUZJY_深圳本地人林女士.docx',
 '2_JNU LLW_DB.docx',
 '2_JNU LLW_LXS.docx',
 '2_JNU LLW_RRAY.docx',
 '2_JNU LLW_SYJZ.docx',
 '2_JNU LLW_XBB.docx',
 '3_SZTUWWX_李先生.docx',
 '3_SZTUWWX_许女士.docx',
 '3_SZTUWWX_谭女士霍先生.docx',
 '3_SZTUWWX_邓先生王女士.docx',
 '3_SZTUWWX_陈女士.docx',
 '4_UT_LJY_UU.docx',
 '4_UT_LJY_WW.docx',
 '4_UT_LJY_XW.docx',
 '5_CS+刘阿姨.docx',
 '5_CS+尤老师.docx',
 '5_CS+钟老师.docx',
 '5_CS+黄先生.docx',
 '6_szulys_nw.docx',
 '6_szulys_乔麦.docx',
 '7_SZUF_LT.docx',
 '7_SZUF_坚.docx',
 '7_SZUF_大花.docx',
 '8_hkust_wdq_q.docx',
 '8_hkust_wdq_yyq.docx',
 '9_SZUCJH_发哥.docx',
 '9_SZUCJH_小贾.docx',
 '9_SZUCJH_小陈.docx',
 '9_SZUCJH_老陈.docx',
 '10_SZTU-lx_何爷爷.docx',
 '10_SZTU-lx_凹凸曼&李先生.docx',
 '11_SZUH_LEI.docx',
 '11_SZUH_YUNWEI.docx',
 '12_SZUL_Roson.docx',
 '12_SZUL_夏天.docx',
 '12_SZUL_张律师.docx',
 '12_SZUL_美丽.docx',
 '12_SZUL_花生.docx',
 '13_SZTULJK_HSF.docx',
 '13_SZTULJK_HYZ.docx',
 '13_SZTULJK_LDH.docx',
 '13_SZTULJK_LJP.docx',
 '14_SZUlsr-jiachunxia.docx',
 '14_SZUlsr-wuxin.docx',
 '15_PKU+DYE_Suning.docx',
 '15_PKU+DYE_Xin.docx',
 '15_PKU+DYE_Xuan.docx',
 '17_SCNUC_小于先生.docx',
 '17_SCNUC_小于女士.docx',
 '18_SZU_XZA_HE.docx',
 '19_sustech_qk_He.docx',
 '19_sustech_qk_meng.docx',
]

In [3]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return ''.join(full_text)

def split_text(text, max_length=15000):
    # 使用\n分割文本为句子列表
    sentences = text.split('\n')
    # display(len(sentences))
    parts = []
    current_part = ""

    for sentence in sentences:
        # 检查加入当前句子是否会超过最大长度
        if len(current_part) + len(sentence) + 1 > max_length:  # +1 是为了计算加入的换行符
            # 如果会超过，当前部分结束，开始新的部分
            parts.append(current_part)
            current_part = sentence  # 开始新的部分，当前句子是新部分的第一句
        else:
            # 如果不会超过，将当前句子加入当前部分
            if current_part:  # 如果当前部分不为空，先加入换行符
                current_part += '\n'
            current_part += sentence

    # 循环结束后，将最后一部分（如果有）加入到部分列表中
    if current_part:
        parts.append(current_part)

    return parts


# POI


In [4]:
df_poi = pd.read_csv('data/output_match/match_frame/POI/深圳市POI_2014.csv',encoding='utf-16',sep='\t')
type_list  = list(df_poi["type"].sort_values().reset_index(drop=True).unique())

# with open("data/output_match/match_frame/POI/type_list.txt", "w") as f:
#     for item in type_list:
#         f.write("%s\n" % item)

# with open("data/output_match/match_frame/POI/selected_type_list.txt", "r") as f:
#     selected_type = f.read().splitlines()

In [5]:
selected_place_type = [ 
    '地名地址信息;地名地址信息;地名地址信息',
    '地名地址信息;普通地名;乡镇级地名',
    '地名地址信息;普通地名;区县级地名',
    '地名地址信息;普通地名;普通地名',
    '地名地址信息;普通地名;村庄级地名',
    '地名地址信息;普通地名;街道级地名',
    '地名地址信息;热点地名;热点地名',
    '地名地址信息;行政地名;乡镇级地名',
    '地名地址信息;行政地名;区县级地名',
    '地名地址信息;行政地名;地市级地名',
    '地名地址信息;行政地名;村庄级地名',
    '地名地址信息;行政地名;村庄级地名|公司企业;工厂;工厂',
    '地名地址信息;行政地名;村庄级地名|商务住宅;产业园区;产业园区',
    '地名地址信息;行政地名;村庄级地名|商务住宅;住宅区;住宅小区',
    '地名地址信息;行政地名;村庄级地名|地名地址信息;交通地名;道路名',
    '地名地址信息;行政地名;村庄级地名|地名地址信息;热点地名;热点地名',
    '地名地址信息;行政地名;村庄级地名|地名地址信息;自然地名;山',
    '地名地址信息;行政地名;村庄级地名|地名地址信息;自然地名;岛屿',
    '地名地址信息;行政地名;村庄级地名|地名地址信息;自然地名;河流',
    '地名地址信息;行政地名;村庄级地名|风景名胜;风景名胜;风景名胜',
    '地名地址信息;行政地名;村庄级地名|餐饮服务;中餐厅;中餐厅',
 ]

# place_type = [i for i in type_list if "地名地址信息" in i[0:6]]

In [6]:
df_poi.head(1)

,id,lng,lat,name,address,telephone,type,areaid,wgslng,wgslat,bdlng,bdlat
0,32132404,114.10965,22.54915,安嘉英,宝安南路2161号,0755-82480288,购物服务;商场;商场,440303,114.104537,22.551834,114.116081,22.555442


In [7]:
df_poi_selected = df_poi.loc[df_poi["type"].isin(selected_place_type)].reset_index(drop=True)
df_poi_selected["method"] = "POI"
df_poi_selected.rename(columns={"name":"location_name"}, inplace=True)

# Concat data from different methods and sources

In [8]:
df_hanlp = pd.read_csv("data/output_match/match_frame/hanlp.csv")
df_hanlp.drop_duplicates(inplace=True)
df_patch = pd.read_csv("data/output_match/match_frame/patch.csv")
df_subway = pd.read_csv("data/output_match/match_frame/subway.csv").groupby("location_name").agg({"type":lambda x:str(list(x))}).reset_index()
df_subway["method"] = "subway"

display(df_hanlp.head(),df_patch.head(),df_subway.head(),df_poi_selected.head(1))

,location_name,method
0,上沙,hanlp
1,东莞,hanlp
2,东门,hanlp
3,东门老街,hanlp
4,丹竹头,hanlp


,location_name,alias,method
0,107国道,NaN,patch
1,11号线,十一号线,patch
2,13号线,十三号线,patch
3,14号线,十四号线,patch
4,16号线,十六号线,patch


,location_name,type,method
0,上塘站,['4'],subway
1,上屋站,['6'],subway
2,上川站,['12'],subway
3,上李朗站,['10'],subway
4,上梅林站,"['4', '9']",subway


,id,lng,lat,location_name,address,telephone,type,areaid,wgslng,wgslat,bdlng,bdlat,method
0,34542766,114.12016,22.541107,国贸,NaN,NaN,地名地址信息;热点地名;热点地名,440303,114.115063,22.543799,114.126576,22.547446,POI


先把hanlp考虑进去，看哪些是hanlp识别不出来的：我们只需要找hanlp识别不出来的

In [9]:
##* 只需要poi的locationname和type，不应该有重复的location_name
df_poi_selected_dd = df_poi_selected.drop_duplicates(subset=['location_name']).reset_index(drop=True)

df_concat = df_hanlp.merge(df_patch,on=["location_name","method"],how="outer").merge(df_subway,on=["location_name","method"],how="outer").merge(df_poi_selected_dd,on=["location_name","method","type"],how="outer")
df_concat.sort_values(by=["location_name","method"],inplace=True)
# df_concat.to_csv("data/output_match/match_frame/concat.csv",index=False,encoding='utf-16',sep='\t')

对于一个地名，hanlp有时识别出来，有时没有识别出来，那么该地名在后续匹配中也不再考虑了。

我们假设上面的现象（没有被hanlp识别出来）的词是少数。因此，如果concat_groupby的表格里有method为hanlp词，那么就不需要在其他method的表格里考虑（应该将他在concat_groupby里删掉）

In [10]:
df_concat_groupby = df_concat.groupby(["location_name"]).agg({"method":lambda x: list(x)}).reset_index()
df_concat_groupby["value"] = df_concat_groupby["method"].apply(lambda x: len(x))

## remove the list containing "hanlp"
df_concat_groupby_nohanlp = df_concat_groupby.loc[df_concat_groupby.apply(lambda x: "hanlp" not in x["method"],axis=1)]

## 找到重复的location_name（method种类大于1）
df_concat_multimethod_nohanlp = df_concat_groupby_nohanlp.loc[(df_concat_groupby_nohanlp["value"]>1)].reset_index(drop=True)#.to_csv("data/output_match/match_frame/concat_multimethod_nohanlp.csv",index=False,encoding='utf-16',sep='\t')
display(df_concat_multimethod_nohanlp)

,location_name,method,value
0,三和,"[POI, patch]",2
1,上梅林,"[POI, patch]",2
2,会展中心站,"[patch, subway]",2
3,华侨城,"[POI, patch]",2
4,南联站,"[patch, subway]",2
5,坪山中心站,"[patch, subway]",2
6,大学城,"[POI, patch]",2
7,大运站,"[patch, subway]",2
8,市民中心站,"[patch, subway]",2
9,景田站,"[patch, subway]",2


发现method的种类大于1的location_name都是在patch里面，所以只用在patch里面删除重复的location_name，重复上述的步骤就好

In [11]:
df_patch_dd = df_patch[df_patch['location_name'].apply(lambda x: x not in df_concat_multimethod_nohanlp["location_name"].values)]

df_concat_final = df_hanlp.merge(df_patch_dd,on=["location_name","method"],how="outer").merge(df_subway,on=["location_name","method"],how="outer").merge(df_poi_selected_dd,on=["location_name","method","type"],how="outer")
df_concat_final.sort_values(by=["location_name","method"],inplace=True)
# display(df_concat_final.head(1))

df_concat_groupby_final = df_concat_final.groupby(["location_name"]).agg({"method":lambda x: list(x)}).reset_index()
df_concat_groupby_final["value"] = df_concat_groupby_final["method"].apply(lambda x: len(x))
## remove the list containing "hanlp"
df_concat_groupby_nohanlp_final = df_concat_groupby_final.loc[df_concat_groupby_final.apply(lambda x: "hanlp" not in x["method"],axis=1)]
# display(df_concat_groupby_nohanlp_final.head(1))

## we need "alias" "method" "type"
# df_concat_groupby_nohanlp_final.drop(columns=["value"]).merge(df_patch.drop(columns=["method"]), on = "location_name", how = "left").merge(df_subway.drop(columns=["method"]), on = "location_name", how = "left").merge(df_poi_selected_dd[["location_name","type"]], on = "location_name", how = "left").to_csv("data/output_match/match_frame/concat_final.csv",index=False,encoding='utf-16',sep='\t')
df_result = df_concat_groupby_nohanlp_final.drop(columns=["value"]).merge(df_patch.drop(columns=["method"]), on = "location_name", how = "left").merge(df_subway.drop(columns=["method"]), on = "location_name", how = "left")

In [12]:

## remove some strange words
remove_words = ["建设","新建","高峰","前进","公安","海湾","楼下","西边","中村"]
df_result_filtered = df_result[~df_result['location_name'].isin(remove_words)].to_csv("data/output_match/match_frame/concat_final.csv",index=False,encoding='utf-16',sep='\t')


# Match

extend the patched location to the list

In [13]:
df_match = pd.read_csv("data/output_match/match_frame/concat_final.csv",encoding='utf-16',sep='\t')
df_match

,location_name,method,alias,type
0,107国道,['patch'],NaN,NaN
1,10号干线,['POI'],NaN,NaN
2,11号线,['patch'],十一号线,NaN
3,13号线,['patch'],十三号线,NaN
4,14号线,['patch'],十四号线,NaN
...,...,...,...,...
3129,龙西村,['POI'],NaN,NaN
3130,龙辉路,['POI'],NaN,NaN
3131,龙过水,['POI'],NaN,NaN
3132,龟咀,['POI'],NaN,NaN


In [14]:
def match_one(my_filepath):
    texts = read_docx(my_filepath)
    # print(texts)
    texts_list = texts.split("\n")
    # display(len(texts_list))

    ner_list = []
    for text in texts_list:
        tmp = []
        for patch in list(df_match["location_name"].values):
            if patch.strip() in text:
                tmp.append([patch.strip(),"ns",0,0])
            # else:
            #     tmp.append([])
        for _, row in df_match.dropna().iterrows():
            for i in row["alias"].split(" "):
                if i.strip() in text:
                    tmp.append(row["location_name"],"ns",0,0)
                    break
        ner_list.append(tmp)
    my_filename = my_filepath.split("/")[-1].split(".")[0]
    return ner_list, my_filename

In [15]:
range_list = range(len(file_list))

# range_list = [1]
for i in tqdm(range_list):
    my_filepath = f"data/Interview transcripts-round 3/{file_list[i]}"
    ner_list, filename = match_one(my_filepath)
    # display(ner_list)
    with open(f"data/output_match/txt/{filename}.txt","w") as f:
        f.write("[\n")
        for item in ner_list:
            f.write("%s\n" % item)
        f.write("]")

100%|██████████| 53/53 [00:31<00:00,  1.69it/s]


将list保存到`txt`中

还需要在txt里筛选一下.....然后更新到remove_words里

# Convert match_list to df

In [16]:
def convert_one_list_todf(my_filename):
    result = get_one_patch_list(my_filename)
    # df = pd.DataFrame(result, columns=['location_name', 'value', 'method']).groupby("location_name").agg({"value": lambda x: list(x)}).reset_index()
    # df["count"] = df["value"].apply(lambda x: len(x))
    df = pd.DataFrame(result, columns=['location_name', 'value', 'method'])
    df["filename"] = my_filename
    df.to_csv(f"data/output_match/csv/{my_filename}.csv", index=False)
    return df

def get_one_patch_list(my_filename):
    with open(f"data/output_match/txt/{my_filename}.txt", "r") as f:
        content = f.read()
        tmp_ner_list = [ast.literal_eval(i.strip()) for i in content[1:-1].strip().split("\n")]
        result =[[item[0], (index, item[2],item[3]),"patch"] for index, sublist in enumerate(tmp_ner_list) for item in sublist if item[1] == 'ns']
    return result

In [ ]:
# df_tmp = convert_one_list_todf("1_SZUZJY_来深33年邹先生")
# df_tmp

In [ ]:
# range_list = range(len(file_list))

# for i in tqdm(range_list):
#     filename = file_list[i].split(".")[0]
#     convert_one_list_todf(filename)